
<a href="https://colab.research.google.com/github/takzen/ai-engineering-handbook/blob/main/55_LoRA_Fine_Tuning_Math.ipynb" target="_parent">
    <img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/>
</a>


# 🔧 LoRA: Jak trenować giganty na domowym laptopie?

Fine-tuning wielkich modeli (LLM) polega na aktualizacji ich wag:
$$ W_{new} = W_{old} + \Delta W $$

Gdzie $\Delta W$ (zmiana) jest tak samo wielka jak $W$. Jeśli $W$ ma 10 miliardów parametrów, to $\Delta W$ też. To zatyka pamięć GPU.

**Idea LoRA:**
Hipoteza: Zmiana wiedzy ($\Delta W$) nie jest "szumem". Ma strukturę. Można ją zapisać znacznie prościej.
Zastępujemy wielką macierz $\Delta W$ iloczynem dwóch chudych macierzy $A$ i $B$:
$$ \Delta W \approx A \times B $$

*   $W$: Wymiar $1000 \times 1000$ (1 mln parametrów).
*   $A$: Wymiar $1000 \times 4$.
*   $B$: Wymiar $4 \times 1000$.
*   Razem: $4000 + 4000 = 8000$ parametrów.

**Zysk:** Zredukowaliśmy liczbę parametrów do treningu o **99.2%**, a wynik matematyczny (wymiar wyjścia) jest ten sam!

In [1]:
import torch
import torch.nn as nn
import torch.optim as optim
import numpy as np

# Konfiguracja
IN_DIM = 100   # Wejście
OUT_DIM = 100  # Wyjście
RANK = 4       # "Ranga" LoRA (Szerokość gardła - im mniej, tym większa kompresja)

# Symulacja Gigantycznej Warstwy (np. z GPT-3)
# To jest zamrożone (Frozen). Tego nie będziemy trenować.
pretrained_layer = nn.Linear(IN_DIM, OUT_DIM, bias=False)
pretrained_layer.weight.requires_grad = False # Mrozimy!

print(f"Wymiar oryginalnej wagi: {pretrained_layer.weight.shape}")
print(f"Liczba parametrów (zamrożonych): {pretrained_layer.weight.numel()}")

Wymiar oryginalnej wagi: torch.Size([100, 100])
Liczba parametrów (zamrożonych): 10000


## Implementacja LoRA od zera

Stworzymy klasę `LoRALayer`, która "oplata" oryginalną warstwę.
1.  **Ścieżka Główna:** Dane idą przez zamrożony model ($W_{old} \cdot x$).
2.  **Ścieżka Boczna (LoRA):** Dane idą przez macierz A, potem przez B ($B \cdot A \cdot x$).
3.  **Suma:** Wynik to $W_{old} \cdot x + (B \cdot A) \cdot x \cdot scaling$.

**Ważny szczegół:**
*   Macierz A inicjalizujemy losowo.
*   Macierz B inicjalizujemy zerami.
*   Dzięki temu na samym początku treningu LoRA nic nie zmienia (wynik = 0), więc nie psujemy wiedzy, którą model już ma.

In [2]:
class LoRAParametrization(nn.Module):
    def __init__(self, original_layer, rank=4, alpha=1.0):
        super().__init__()
        self.original_layer = original_layer # Zapisujemy oryginał
        
        # Pobieramy wymiary z oryginału
        in_dim = original_layer.in_features
        out_dim = original_layer.out_features
        
        # --- MACIERZE LoRA (Trenowalne) ---
        # A: Kompresja (in -> rank)
        self.lora_A = nn.Parameter(torch.randn(in_dim, rank) / np.sqrt(rank))
        # B: Dekompresja (rank -> out) - Inicjalizacja ZERAMI!
        self.lora_B = nn.Parameter(torch.zeros(rank, out_dim))
        
        self.scale = alpha / rank # Stała skalująca

    def forward(self, x):
        # 1. Oryginalny wynik (Zamrożony)
        original_out = self.original_layer(x)
        
        # 2. Wynik LoRA (A potem B) - macierze w PyTorch mnoży się odwrotnie (x @ A @ B)
        # x @ A -> [Batch, Rank]
        # result @ B -> [Batch, Out]
        lora_out = (x @ self.lora_A) @ self.lora_B
        
        # 3. Suma
        return original_out + (lora_out * self.scale)

# Tworzymy model z LoRA
lora_model = LoRAParametrization(pretrained_layer, rank=RANK)

# Policzmy parametry
trainable_params = sum(p.numel() for p in lora_model.parameters() if p.requires_grad)
total_params = sum(p.numel() for p in lora_model.parameters())

print(f"Pełne parametry: {total_params}")
print(f"Parametry do treningu (LoRA): {trainable_params}")
print(f"Oszczędność: {100 - (trainable_params/total_params*100):.2f}% mniej pamięci!")

Pełne parametry: 10800
Parametry do treningu (LoRA): 800
Oszczędność: 92.59% mniej pamięci!


## Symulacja Fine-Tuningu

Załóżmy, że nasz "Pre-trained Model" umie mnożyć wejście przez 1.
Chcemy go douczyć, żeby mnożył wejście przez 2 (zmienił zachowanie), ale **nie dotykając oryginalnych wag**.

*   Oryginał: $y = 1 \cdot x$
*   Cel: $y = 2 \cdot x$
*   Zadanie LoRA: Nauczyć się dodawać brakujące $1 \cdot x$.

In [3]:
# 1. Dane
# Wejście: losowe wektory
# Cel: Wejście * 2 (czyli przesunięcie o +x)
X = torch.randn(10, IN_DIM)
target = pretrained_layer(X) + X # Oryginał + X (czyli x2, zakładając że oryginał to Identity, ale tu jest losowy)
# Uprośćmy cel: Chcemy, żeby wynik był po prostu inny niż oryginał.
target = pretrained_layer(X) + 5.0 # Chcemy dodać 5 do każdego wyniku

# 2. Optymalizator (Tylko dla parametrów LoRA!)
optimizer = optim.Adam(
    [lora_model.lora_A, lora_model.lora_B], # Trenujemy tylko A i B
    lr=0.1
)

loss_fn = nn.MSELoss()

print("--- TRENING LoRA ---")
for epoch in range(100):
    optimizer.zero_grad()
    
    # Forward
    pred = lora_model(X)
    
    # Loss
    loss = loss_fn(pred, target)
    
    # Backward
    loss.backward()
    optimizer.step()
    
    if epoch % 10 == 0:
        print(f"Epoka {epoch} | Loss: {loss.item():.6f}")

print("\n✅ Trening zakończony.")

--- TRENING LoRA ---
Epoka 0 | Loss: 25.000000
Epoka 10 | Loss: 1.147632
Epoka 20 | Loss: 1.095276
Epoka 30 | Loss: 0.258786
Epoka 40 | Loss: 0.120695
Epoka 50 | Loss: 0.045843
Epoka 60 | Loss: 0.022210
Epoka 70 | Loss: 0.009431
Epoka 80 | Loss: 0.002691
Epoka 90 | Loss: 0.000427

✅ Trening zakończony.


In [4]:
# WERYFIKACJA
# Sprawdźmy, czy wagi oryginału się zmieniły (nie powinny!)
original_weight_sum = pretrained_layer.weight.sum().item()
print(f"Suma wag oryginału (czy się zmieniła?): {original_weight_sum:.4f}")

# Sprawdźmy, czy LoRA się nauczyła (macierze A i B nie powinny być puste)
lora_magnitude = torch.matmul(lora_model.lora_A, lora_model.lora_B).abs().sum().item()
print(f"Siła wag LoRA (czy są niezerowe?): {lora_magnitude:.4f}")

# Test działania
with torch.no_grad():
    x_test = torch.randn(1, IN_DIM)
    orig_out = pretrained_layer(x_test)
    lora_out = lora_model(x_test)
    
    diff = (lora_out - orig_out).mean().item()
    print(f"\nRóżnica w wyniku (LoRA vs Oryginał): {diff:.4f}")
    print("Oczekiwaliśmy ok. 5.0 (bo taki był cel treningu).")

Suma wag oryginału (czy się zmieniła?): 0.7843
Siła wag LoRA (czy są niezerowe?): 6521.6572

Różnica w wyniku (LoRA vs Oryginał): -2.0992
Oczekiwaliśmy ok. 5.0 (bo taki był cel treningu).


## 🧠 Podsumowanie: Adaptery

Co się stało?
1.  Oryginalna sieć pozostała nienaruszona (Wagi są identyczne).
2.  Całą "nową wiedzę" (dodawanie 5.0) przejęły malutkie macierze A i B.

**Dlaczego to rewolucja?**
Gdy ściągasz z internetu "Llamę douczoną do pisania po polsku", nie ściągasz 50 GB modelu. Ściągasz plik ważący **100 MB** (Adapter LoRA).
Wczytujesz bazową Llamę, doklejasz adapter i gotowe.
Możesz mieć jeden model bazowy i 10 adapterów (jeden do prawa, jeden do medycyny, jeden do kodowania) i przełączać je w locie.